In [2]:
#! usr/bin/env python3
import praw
import pandas as pd
import datetime as dt

In [3]:
reddit = praw.Reddit(client_id='4qQrwmPQAVOwiw', \
                     client_secret='MVf28OrOH-Ak_-XLSlLHWqbfono', \
                     user_agent='API dan', \
                     username='blablabla123983', \
                     password='newguy')

In [4]:
subreddit = reddit.subreddit('Bitcoin')

In [5]:
top_subreddit = subreddit.top()

In [6]:
top_subreddit = subreddit.top(limit=500)

In [7]:
for submission in subreddit.top(limit=1):
    print(submission.title, submission.id)

It's official! 1 Bitcoin = $10,000 USD 7g9cd3


In [17]:
topics_dict = { "title":[], \
                "score":[], \
                "id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "body":[]}


In [18]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [20]:
topics_data = pd.DataFrame(topics_dict)
topics_data

,body,comms_num,created,id,score,title,url
0,,4595,1.511948e+09,7g9cd3,48506,"It's official! 1 Bitcoin = $10,000 USD",https://v.redd.it/e7io27rdgt001
1,,791,1.517741e+09,7v438b,48480,The last 3 months in 47 seconds.,https://v.redd.it/typ8fdslz3e01
2,,3172,1.511701e+09,7fkqh5,42431,It's over 9000!!!,https://i.imgur.com/jyoZGyW.gifv
3,,804,1.515300e+09,7olruz,42055,Everyone who's trading BTC right now,http://cdn.mutually.com/wp-content/uploads/201...
4,,1243,1.512064e+09,7gla05,29318,I hope James is doing well,https://i.redd.it/h4ngqma643101.jpg
5,,1531,1.516123e+09,7qr4ym,29133,Weeeeeeee!,https://i.redd.it/iwl7vz69cea01.gif
6,,1180,1.517435e+09,7u9zbd,28322,Bitcoin.. The King,https://i.redd.it/4tl0oustqed01.jpg
7,,1321,1.513181e+09,7jht7c,28060,Nothing can increase by that much and still be...,https://i.imgur.com/oWePY7q.jpg
8,,918,1.511001e+09,7dpypn,26952,"This is why I want bitcoin to hit $10,000",https://i.redd.it/fhzsxgcv9nyz.jpg
9,,1505,1.518938e+09,7yaaz2,26327,Bitcoin Doesn't Give a Fuck.,https://v.redd.it/ty2y74gawug01


In [21]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [22]:
_timestamp = topics_data["created"].apply(get_date)

In [23]:
topics_data = topics_data.assign(timestamp = _timestamp)

In [26]:
topics_data.to_csv('bitcoindatanew.csv', index=False) 

In [25]:
topics_data

,body,comms_num,created,id,score,title,url,timestamp
0,,4595,1.511948e+09,7g9cd3,48506,"It's official! 1 Bitcoin = $10,000 USD",https://v.redd.it/e7io27rdgt001,2017-11-29 10:25:24
1,,791,1.517741e+09,7v438b,48480,The last 3 months in 47 seconds.,https://v.redd.it/typ8fdslz3e01,2018-02-04 11:42:03
2,,3172,1.511701e+09,7fkqh5,42431,It's over 9000!!!,https://i.imgur.com/jyoZGyW.gifv,2017-11-26 13:55:02
3,,804,1.515300e+09,7olruz,42055,Everyone who's trading BTC right now,http://cdn.mutually.com/wp-content/uploads/201...,2018-01-07 05:38:56
4,,1243,1.512064e+09,7gla05,29318,I hope James is doing well,https://i.redd.it/h4ngqma643101.jpg,2017-11-30 18:50:46
5,,1531,1.516123e+09,7qr4ym,29133,Weeeeeeee!,https://i.redd.it/iwl7vz69cea01.gif,2018-01-16 18:13:04
6,,1180,1.517435e+09,7u9zbd,28322,Bitcoin.. The King,https://i.redd.it/4tl0oustqed01.jpg,2018-01-31 22:46:59
7,,1321,1.513181e+09,7jht7c,28060,Nothing can increase by that much and still be...,https://i.imgur.com/oWePY7q.jpg,2017-12-13 17:02:51
8,,918,1.511001e+09,7dpypn,26952,"This is why I want bitcoin to hit $10,000",https://i.redd.it/fhzsxgcv9nyz.jpg,2017-11-18 11:25:57
9,,1505,1.518938e+09,7yaaz2,26327,Bitcoin Doesn't Give a Fuck.,https://v.redd.it/ty2y74gawug01,2018-02-18 08:19:24


In [ ]:
import logging
import settings
#CLI input
import argparse
import time

#reddit api (Python Reddit API Wrapper)
import praw
import os

import datetime

#JSON parsing
import json

import time
#Multithreading and CSV processing 
from multiprocessing import Process
import csv

#Setting default to UTF8 to deal with pesky ascii errors in python 2.x
import sys
reload(sys)
sys.setdefaultencoding("utf8")

#Change this to CSV writer
def saveSubmission(submission, filename):
    #Reddit accounts can be deleted, and then the username won't show. If it doesn't exist fill in [deleted]
    if not submission.author:
        name = '[deleted]'
    #Otherwise just assign the author name 
    else:
        name = submission.author.name

    #Write it to a CSV 
    csvwriter = csv.writer(open(filename, "a"))
    csvwriter.writerow([datetime.datetime.fromtimestamp(submission.created).strftime('%c'), submission.permalink,  name, submission.title, submission.selftext, submission.score])

# Downloads all the self posts from given subreddit
# ts_interval - the interval in seconds for each request cycle
# largest_timestamp - if not set, is set to 12 hours from now.
def getThreads(subredditName, ts_interval, ep1, ep2, user_ag, user):
    #Set filename for this subreddit. Timestamp it so everytime this script is run, results are appended to differnt file 
    filename = 'Reddit_'+subredditName+'_'+str(int(time.time()))+'.csv'

    #Creating header rows in the CSV file 
    csvwriter = csv.writer(open(filename, "a"))
    csvwriter.writerow(['timestamp', 'url',  'author', 'postTitle', 'postContent', 'score'])
    #Initialize the Reddit client with client ID, secret key, user agent (arbitrary), password and username
    #See the included readme for detailed explanation 
    reddit = praw.Reddit(client_id=user['clientID'],client_secret=user['secretKey'],user_agent=user_ag,password=user['password'],username=user['username'])
    print(reddit.user.me())
    #If no time epoch is specified, 
    if ep2 is None:
        ep2 = int(time.time()) + 12*3600

    #cts for current timestamp. Defines the sliding window between which the search query is executed. 
    cts2 = ep2
    #Substract the ts interval from the second timstamp to move the window
    cts1 = ep2 - ts_interval
    currentTSInterval = ts_interval
    while True:
        #Use reddit search API to search for posts between two timestamps (created above), on the subreddit defined in subRedditName
        #Using cloudsearch as syntax, because otherwise it is not possible to search between timestamps 
        try:
            searchResults = list(reddit.subreddit(subredditName).search('timestamp:{}..{}'.format(cts1, cts2), syntax='cloudsearch'))
        #If it errors for whatever reason, log the error and continue 
        except Exception as e:
            logging.exception(e)
            continue

        #logging.debug("Got {} submissions in interval {}..{}".format(len(searchResults), cts1, cts2))
        #It returns at most 100 search results, so if it maxes out, we split the interval and search again 
        #This way we ensure that we do not miss any posts 
        if len(searchResults) == 100:
            currentTSInterval /= 2
            cts1 = cts2 - currentTSInterval
           # logging.debug("Reducing time interval to {}".format(currentTSInterval))
            continue

        #Printing Reddit post title to show progress 
        for submission in searchResults:
            print (submission.title)
            saveSubmission(submission, filename)

        #Move the timestamp window by setting the second ts to the first, and decreasing the second one
        cts2 = cts1
        cts1 = max(ep1, cts2 - currentTSInterval)

        #Once the timestamps meet, the range is effectively closed and no more topics are to be collected 
        if cts1 == cts2:
            break

        if len(searchResults) <= 7:
            currentTSInterval *= 2
           # logging.debug("Increasing time interval to {}".format(currentTSInterval))

def main():
    parser = argparse.ArgumentParser(description='Parallel Subreddit Downloader')
    parser.add_argument("--subreddit", dest="subreddit", required=True, help="Download the whole subreddit")
    parser.add_argument("--timestamp_interval", dest="timestamp_interval", type=int, required=True)
    parser.add_argument("--year", dest="collect_year", type=int, required=True)
    argsx = parser.parse_args()
    collect_year = str(argsx.collect_year)


    #Timeframe between which to scrape
    tst1 = '01.01.'+collect_year+' 00:00:00'
    tst2 = '31.12.'+collect_year+' 23:59:59'
    pattern = '%d.%m.%Y %H:%M:%S'

    #Timezone definition, OS based
    os.environ['TZ'] = 'UTC'

    #Convert time string to epoch 
    epoch1 = int(time.mktime(time.strptime(tst1, pattern)))
    epoch2 = int(time.mktime(time.strptime(tst2, pattern)))

    #Split the increments over 8 different multithreading processes for increased speed 
    inc = (epoch2 - epoch1 + 1)/8
    ep1 = [0]*8
    ep2 = [0]*8
    processes = []

    for i in range(0, 8):
        ep1[i] = epoch1 + i*inc
        ep2[i] = epoch1 + (i + 1)*inc - 1
        user_agx = 'Collection' + str(i)
        user = settings.config
        p = Process(target=getThreads, args=(argsx.subreddit, argsx.timestamp_interval, ep1[i], ep2[i], user_agx, user))
        p.start()
        processes.append(p)

if __name__ == "__main__":
	start = time.time()
	main()
	end = time.time()
	print (end - start)
	


In [19]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

h2o.init()

data = pd.read_csv("/Users/Daniel/Documents/Courses_HEC/Business Case eleven/Bitcoin-NLP-Strategy/Data/20180920_mastertable.csv")
invest = data.invest
date = data.date
data = data.dropna()
data_scaled = data.drop(["date","Unnamed: 0","return_day+1","invest"], axis=1)
cols = data_scaled.columns
data_scaled = preprocessing.scale(data_scaled)
data_scaled = pd.DataFrame(data_scaled)
data_scaled.columns = cols
data_scaled["invest"] = invest
data_scaled["date"] = date
data = data_scaled
data

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 2 mins
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,19 days
H2O cluster name:,Daniel
H2O cluster total nodes:,1
H2O cluster free memory:,1.807 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


,close,volume,volatility_14,googletrends_buy_sell,Tweets (#),Active Influencers (#),Twitter Average SA,Forum SA Merit,Forum SA Merit (weighted),Reddit Comments (#),Reddit Average SA,invest,date
0,-1.259410,-0.885228,-1.815638,-0.617938,-1.229630,-1.308931,2.019758,-0.730275,1.544647,-0.857860,1.610330,True,2017-05-01
1,-1.246628,-0.809811,-1.826866,1.350456,-1.368504,-0.838780,4.091047,-0.024282,1.840794,-0.987301,1.935256,True,2017-05-02
2,-1.249417,-0.566294,-1.803027,-0.836648,-1.185774,-1.120870,1.869204,0.397697,2.109174,-0.770868,0.690981,False,2017-05-03
3,-1.263814,-0.435161,-1.564961,-1.602135,-1.448905,-1.026840,0.465855,0.413071,1.449825,-0.811220,0.253766,False,2017-05-04
4,-1.250772,-0.672864,-1.518604,0.326891,-1.602398,-1.967142,-0.707008,-1.211215,1.046577,-0.925463,-0.907606,True,2017-05-05
5,-1.245550,-0.477814,-1.524305,1.153617,-1.719345,-2.343263,-0.836226,0.580530,1.699751,-0.995162,0.021910,True,2017-05-06
6,-1.225198,-0.364056,-1.440510,-0.836648,-1.193084,-1.402961,0.895888,1.616628,2.098285,-0.897164,0.622462,True,2017-05-07
7,-1.212624,-0.464677,-1.427080,0.858358,-1.112683,-1.026840,-0.024230,-0.211321,1.233249,-0.688591,0.362868,True,2017-05-08
8,-1.202752,-0.711929,-1.425187,2.225298,-1.098064,-0.932810,-0.014758,2.317732,1.980982,-0.714794,-0.162762,True,2017-05-09
9,-1.188450,-0.635890,-1.416988,-0.414311,-1.229630,-1.026840,-0.396239,0.478663,1.295990,-0.676014,0.871596,True,2017-05-10


In [ ]:
for lines in range(12):
        data["lag_return_day"+str(lines+1)]=data["return_day+1"].shift(lines+1)
        data["lag_close_day"+str(lines+1)]=data["close"].shift(lines+1)
        data["lag_volume_day"+str(lines+1)]=data["volume"].shift(lines+1)
        data["lag_googletrends_buy_sell_day"+str(lines+1)]=data["googletrends_buy_sell"].shift(lines+1)
        data["lag_Tweets (#)"+str(lines+1)]=data["Tweets (#)"].shift(lines+1)
        data["lag_Active Influencers (#)"+str(lines+1)]=data["Active Influencers (#)"].shift(lines+1)
        data["lag_Twitter Average SA"+str(lines+1)]=data["Twitter Average SA"].shift(lines+1)
        data["lag_Forum SA Merit (weighted)"+str(lines+1)]=data["Forum SA Merit (weighted)"].shift(lines+1)
        data["lag_Reddit Comments (#)"+str(lines+1)]=data["Reddit Comments (#)"].shift(lines+1)
        data["lag_Reddit Average SA"+str(lines+1)]=data["Reddit Average SA"].shift(lines+1)
        data["lag_Reddit Average SA"+str(lines+1)]=data["Reddit Average SA"].shift(lines+1)

In [15]:
train = data[0:300]
test = data[301:]
#train["invest"] = asfactor(train["invest"])
#test["invest"] = asfactor(test["invest"])
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
x = train.columns
y = "invest"
x.remove(y)

In [17]:
# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 30)
aml.train(x = x, y = y,
          training_frame = train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb
h2o.shutdown(prompt=False)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,mean_per_class_error,rmse,mse
GLM_grid_0_AutoML_20180920_161805_model_0,0.628859,0.666799,0.495283,0.486474,0.236657
DeepLearning_grid_0_AutoML_20180920_161805_model_1,0.589694,0.731078,0.495283,0.511356,0.261485
DeepLearning_grid_0_AutoML_20180920_161805_model_2,0.580975,0.871138,0.481132,0.542005,0.293769
DeepLearning_0_AutoML_20180920_161805,0.55546,0.751228,0.495283,0.519516,0.269897
GBM_grid_0_AutoML_20180920_161805_model_5,0.551172,0.693741,0.5,0.499758,0.249759
DRF_0_AutoML_20180920_161805,0.54306,0.915344,0.5,0.528553,0.279368
GBM_grid_0_AutoML_20180920_161805_model_14,0.529302,0.68678,0.5,0.496842,0.246852
GBM_grid_0_AutoML_20180920_161805_model_3,0.514008,0.769412,0.5,0.52968,0.280561
StackedEnsemble_BestOfFamily_0_AutoML_20180920_161805,0.513722,0.686921,0.5,0.496885,0.246895
XRT_0_AutoML_20180920_161805,0.508934,0.779555,0.480203,0.531886,0.282903
